<a href="https://colab.research.google.com/github/beBijayeeni/Indian-Agricultural-Productivity-Analysis/blob/main/IndianAgriculturalProductivityAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
DATA_SOURCE = "https://www.kaggle.com/datasets/akshatgupta7/crop-yield-in-indian-states-dataset"
print("Data Source:", DATA_SOURCE)

**About Dataset**

This dataset encompasses agricultural data for multiple crops cultivated across various states in India from the year 1997 till 2020. The dataset provides crucial features related to crop yield prediction, including crop types, crop years, cropping seasons, states, areas under cultivation, production quantities, annual rainfall, fertilizer usage, pesticide usage, and calculated yields.

**Columns Description:**

1. **Crop**: The name of the crop cultivated.
2. **Crop_Year**: The year in which the crop was grown.
3. **Season**: The specific cropping season (e.g., Kharif, Rabi, Whole Year).
4. **State**: The Indian state where the crop was cultivated.
5. **Area**: The total land area (in hectares) under cultivation for the specific crop.
6. **Production**: The quantity of crop production (in metric tons).
7. **Annual_Rainfall**: The annual rainfall received in the crop-growing region (in mm).
8. **Fertilizer**: The total amount of fertilizer used for the crop (in kilograms).
9. **Pesticide**: The total amount of pesticide used for the crop (in kilograms).
10. **Yield**: The calculated crop yield (production per unit area).


In [ ]:
#Import data from Google Drive. First mount it
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/DA-LAB/crop_yield.csv')
df.head()

In [ ]:
!pip -q install ydata-profiling
from ydata_profiling import ProfileReport

profile = ProfileReport(df, title="Crop Yield Dataset — Profile Report", explorative=True)
profile.to_file("profile_report.html")
print("Saved: profile_report.html")

In [ ]:
df.shape

In [ ]:
# Remove duplicates
before = df.shape[0]
df = df.drop_duplicates()
after = df.shape[0]
print(f"Removed {before - after} duplicate rows")

In [ ]:
# Check info and missing values
df.info()

In [ ]:
df.isnull().sum()

### Missing Value Analysis
`df.isnull().sum()` shows that all columns have 0 missing values.
Therefore no imputation operation is required in this dataset.
We proceed directly to outlier handling and encoding steps.


In [ ]:
print("Mean of Production:", df['Production'].mean())
print("Median of Production:", df['Production'].median())
print("Mode of Production:", df['Production'].mode()[0])

Mean >> Median >> Mode

The mean is much higher than the median, which usually indicates a right-skewed distribution (a few very large values are pulling the mean up).

The median (13,804) is much smaller than the mean, meaning that half the data points have production less than about 13,800.

The mode is 0, meaning that the most frequent production value is zero — so there are many entries where production was zero or no production recorded.

In [ ]:
# Distribution plot of Production
import seaborn as sns
import matplotlib.pyplot as plt
sns.histplot(df['Production'], bins=30, kde=True)
plt.title("Distribution of Production")
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create a copy of the DataFrame to preserve the original
df_encoded = df.copy()

# Identify categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns

# Apply Label Encoding
le = LabelEncoder()
for col in categorical_cols:
    df_encoded[col] = le.fit_transform(df_encoded[col])

# Confirm encoding
print("Encoded DataFrame:")
df_encoded.head()


In [ ]:
cols = df_encoded.columns
skew_table = df_encoded[cols].skew().sort_values(ascending=False)
print("Skewness of columns:\n", skew_table)

### Skewness Interpretation
Skewness values show extremely high right skew for `Pesticide (25.63)`, `Area (21.85)`, `Production (19.29)`, `Fertilizer (13.41)` and `Yield (12.78)`.  
This means most observations have very small values while few very large values stretch the tail.

Such heavy right skew often occurs in agricultural datasets because only few states/crops have massive scale production while majority are small farmers.

Since tree based models are robust to skewness, transformation is optional for them.
But PCA + linear models + KNN benefit from normalization + scaling before modeling.


In [ ]:
# Compute correlation matrix
corr_matrix = df_encoded.corr()
print(corr_matrix)
# Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
# Outlier counts per continuous column (IQR rule)
cont_cols = df_encoded.columns
outlier_summary = {}
for col in cont_cols:
    Q1, Q3 = df_encoded[col].quantile(0.25), df_encoded[col].quantile(0.75)
    IQR = Q3 - Q1
    lb, ub = Q1 - 1.5*IQR, Q3 + 1.5*IQR
    outlier_summary[col] = int(((df_encoded[col] < lb) | (df_encoded[col] > ub)).sum())
print("Outlier counts (IQR):", outlier_summary)

In [ ]:
plt.figure(figsize=(15, 10))
for i, col in enumerate(df_encoded.columns):
    plt.subplot(2,5, i + 1)
    sns.boxplot(y=df_encoded[col])
    plt.title(f'Box plot of {col}')
plt.tight_layout()
plt.show()

In [ ]:
# Create a copy to avoid modifying the original DataFrame
df_no_outliers = df_encoded.copy()
print(f"Shape before removing outliers: {df_no_outliers.shape}")
# Identify numerical columns again (after encoding)
#numerical_cols = df_no_outliers.select_dtypes(include=['int64', 'float64']).columns

# Remove outliers for each numerical column using IQR
for col in df_no_outliers:
    Q1 = df_no_outliers[col].quantile(0.25)
    Q3 = df_no_outliers[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_no_outliers = df_no_outliers[(df_no_outliers[col] >= lower_bound) & (df_no_outliers[col] <= upper_bound)]

# Show result
print(f"Shape after removing outliers: {df_no_outliers.shape}")

In [ ]:
plt.figure(figsize=(15, 10))
for i, col in enumerate(df_no_outliers):
    plt.subplot(2,5, i + 1)
    sns.boxplot(y=df_no_outliers[col])
    plt.title(f'Box plot of {col}')
plt.tight_layout()
plt.show()

In [ ]:
# Recalculate correlation matrix without outliers
corr_matrix_no_outliers = df_no_outliers.corr()
print("Correlation Matrix (No Outliers):")
print(corr_matrix_no_outliers)
# Plot heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix_no_outliers, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap (No Outliers)")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

# Separate features and target
X = df_no_outliers.drop('Production', axis=1)
y = df_no_outliers['Production']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

# Separate features (X) and target (y)
X = df_no_outliers.drop('Production', axis=1)
y = df_no_outliers['Production']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the models to be evaluated
models = {
    "Random Forest": RandomForestRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42),
    "AdaBoost": AdaBoostRegressor(random_state=42),
    "K-Nearest Neighbors": KNeighborsRegressor()
}

# Dictionary to store results
results = {}

# Loop through the models, train, and evaluate
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    results[name] = [r2, rmse, mae]
    print(f"--- {name} ---")
    print(f"R² Score: {r2:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}\n")

# Model Performance Interpretation

---

### Random Forest
- **R² Score:** 0.96  
- **RMSE:** 1437.29  
The Random Forest model performs very well, explaining 96% of the variance in the target variable. The RMSE value indicates a relatively low average prediction error, showing that this model provides accurate predictions.

---

### XGBoost
- **R² Score:** 0.96  
- **RMSE:** 1405.23  
XGBoost also achieves an excellent performance, matching the Random Forest in R² and slightly outperforming it in RMSE. This suggests that XGBoost has a slightly better fit and lower prediction error than Random Forest.

---

### AdaBoost
- **R² Score:** 0.72  
- **RMSE:** 3658.51  
AdaBoost’s performance is noticeably lower than Random Forest and XGBoost. An R² of 0.72 means it explains 72% of the variance, which is decent but less reliable. The higher RMSE indicates larger average prediction errors.

---

### K-Nearest Neighbors (KNN)
- **R² Score:** 0.34  
- **RMSE:** 5641.49  
KNN shows the weakest performance with only 34% of the variance explained. The high RMSE reflects significant errors in predictions, suggesting KNN is not well-suited for this problem.

---

## Summary:
- **Random Forest** and **XGBoost** are the best-performing models, with very high R² scores and low RMSE values.  
- **AdaBoost** is moderately effective but less accurate.  
- **K-Nearest Neighbors** performs poorly for this task and likely should not be used in production.

Based on these metrics, **XGBoost** might be preferred for slightly better accuracy.


In [ ]:
from sklearn.preprocessing import (
    MinMaxScaler, StandardScaler, RobustScaler, Normalizer,
    QuantileTransformer, PowerTransformer
)
from sklearn.pipeline import Pipeline

# Define the scalers
scalers = {
    "Min-Max": MinMaxScaler(),
    "Z-Score (Standard)": StandardScaler(),
    "Robust Scaling": RobustScaler(),
    "L2 Normalization": Normalizer(norm='l2'),
    "Quantile Uniform": QuantileTransformer(output_distribution='uniform'),
    "Quantile Normal": QuantileTransformer(output_distribution='normal'),
    "Power Transformer": PowerTransformer(method='yeo-johnson'),
    "Log Transform": 'log'
}

# Store results for scaling comparison
scaling_results = {}

# Choose the best model
best_model = XGBRegressor(random_state=42)

# Loop through each scaler
for name, scaler in scalers.items():
    if name == "Log Transform":
        # Apply log transform directly
        X_train_log = np.log1p(X_train)
        X_test_log = np.log1p(X_test)
        best_model.fit(X_train_log, y_train)
        y_pred = best_model.predict(X_test_log)
    else:
        # Create a pipeline with the scaler and the model
        pipeline = Pipeline([
            ('scaler', scaler),
            ('model', best_model)
        ])
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)

    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    scaling_results[name] = [r2, rmse]

# Add the baseline (no scaling) result for comparison
scaling_results["No Scaling (Baseline)"] = results["XGBoost"][:2]

# Display results
scaling_results_df = pd.DataFrame(scaling_results, index=['R² Score', 'RMSE']).T.sort_values(by='R² Score', ascending=False)
scaling_results_df

# Effect of Different Scaling Methods on Model Performance
- All scaling methods except **L2 Normalization** result in the **same R² score (0.959)** and **RMSE (~1405.23)**, indicating no significant difference in model performance across these scaling techniques.


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X_full = df_encoded.drop(columns=['Production'])  # target remains 'Production'
y_full = df_encoded['Production']

X_scaled = StandardScaler().fit_transform(X_full)  # PCA prefers standardized data
pca = PCA(n_components=0.95, svd_solver='full')   # keep 95% variance
X_pca = pca.fit_transform(X_scaled)
print("PCA components:", pca.n_components_, "Explained variance:", pca.explained_variance_ratio_.sum())

In [ ]:
from sklearn.model_selection import cross_val_score, KFold

# Cross-validation on best model
cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_rmse = (-cross_val_score(models["XGBoost"], X, y, scoring='neg_root_mean_squared_error', cv=cv)).mean()
cv_r2 = (cross_val_score(models["XGBoost"], X, y, scoring='r2', cv=cv)).mean()
print(f"XGB 5-fold CV — RMSE: {cv_rmse:.2f}, R²: {cv_r2:.3f}")


In [ ]:
knn_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsRegressor())
])

knn_pipeline.fit(X_train, y_train)
knn_pred = knn_pipeline.predict(X_test)

knn_r2 = r2_score(y_test, knn_pred)
knn_rmse = np.sqrt(mean_squared_error(y_test, knn_pred))
knn_mae = mean_absolute_error(y_test, knn_pred)

print("KNN with StandardScaler (Normalized)")
print("R2:", knn_r2)
print("RMSE:", knn_rmse)
print("MAE:", knn_mae)


In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import numpy as np

models_compare = {}

# Separate features and target for the data without outliers
X_no_outliers = df_no_outliers.drop('Production', axis=1)
y_no_outliers = df_no_outliers['Production']

# XGB Original (using data without outliers)
X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(X_no_outliers, y_no_outliers, test_size=0.2, random_state=42)
xgb_orig = XGBRegressor(random_state=42)
xgb_orig.fit(X_train_orig, y_train_orig)
pred_orig = xgb_orig.predict(X_test_orig)
models_compare["XGB Original"] = [
    r2_score(y_test_orig, pred_orig),
    np.sqrt(mean_squared_error(y_test_orig, pred_orig)),
    mean_absolute_error(y_test_orig, pred_orig)
]

# PCA on the data without outliers
X_scaled_no_outliers = StandardScaler().fit_transform(X_no_outliers)
pca = PCA(n_components=0.95, svd_solver='full')
X_pca_no_outliers = pca.fit_transform(X_scaled_no_outliers)
print("PCA components:", pca.n_components_, "Explained variance:", pca.explained_variance_ratio_.sum())

# Train-test split for PCA data (from data without outliers)
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca_no_outliers, y_no_outliers, test_size=0.2, random_state=42)

# XGB PCA
xgb_pca = XGBRegressor(random_state=42)
xgb_pca.fit(X_train_pca, y_train_pca)
pred_pca = xgb_pca.predict(X_test_pca)
models_compare["XGB PCA"] = [
    r2_score(y_test_pca, pred_pca),
    np.sqrt(mean_squared_error(y_test_pca, pred_pca)),
    mean_absolute_error(y_test_pca, pred_pca)
]

compare_df = pd.DataFrame(models_compare, index=["R2","RMSE","MAE"]).T
compare_df

The XGBoost model trained on the original feature space performed better (R² = 0.959, RMSE ≈ 1405) compared to the XGBoost model trained after PCA dimensionality reduction (R² = 0.934, RMSE ≈ 1775). Although PCA successfully reduced dimensionality while retaining ~99% variance, tree-based models like XGBoost inherently handle nonlinear feature interactions and high dimensional splits effectively, which means they do not significantly benefit from PCA transformation. PCA slightly reduced interpretability and caused information compression which led to reduction in prediction performance.

**Final Conclusion:**

In this analysis, multiple regression models were evaluated for agricultural crop production prediction. Among all tested models, XGBoost emerged as the best performing model with very high predictive accuracy (R² = 0.959 and RMSE ≈ 1405). PCA was also applied to reduce dimensionality and retain 95% variance; however, the XGBoost model trained on PCA-transformed features performed comparatively lower (R² = 0.934) than the model trained on the original dataset. This indicates that tree-based ensemble models like XGBoost already learn hierarchical feature splits effectively and do not require dimensionality reduction for performance enhancement. Therefore, the XGBoost model without PCA is selected as the final recommended regression model for crop prediction.